<a href="https://colab.research.google.com/github/burakericok/hard_sphere_transitions/blob/master/transitions_hard_spheres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transitions Between Critical Points of the Configuration Spaces of Hard Spheres**

This is a notebook that allows one to visualize the precomputed transitions between the critical points of the configuration spaces of hard spheres. The transitions are computed using _the zero temperature string_ method proposed by REF. Currently, the transitions are available for $n=2$ spheres.

Note: This notebook can be used in conjunction with the notebook that plots the known critical points of the configuration spaces of hard spheres.

## **Security Statement**
Connecting to a Jupyter notebook server running on your local machine can provide many benefits. With these benefits come serious potential risks. By connecting to a local runtime, you are allowing the Colaboratory frontend to execute code in the notebook using the local resources on your machine. This means that the notebook could:

- Invoke arbitrary commands (e.g. "rm -rf /")
- Access the local file system
- Run malicious content on your machine

Before attempting to connect to a local runtime, make sure you trust the authors of the notebook and ensure you understand what code is being executed. For more information on the Jupyter notebook server's security model, consult [Jupyter's documentation](https://jupyter-notebook.readthedocs.io/en/stable/security.html).

## **Usage**
The capabilities of this notebook are contained in two functions, `import_database` and `plot_transition`. These are defined in the `Define necessary functions` cell (double clicking expands the cell if you would like to examine the code). Run this cell first.

The next cell imports the database of transition between the critical points for the desired number of spheres, passed as the single argument to `import_database`. The transition databases are available for $n=2$. 

The next cell displays the transition between the selected critical points, passed as the second and the third argument to `plot_transition`. The values of these argument must be integers.

For example, to plot the transition between the first and the third critical point for two spheres:
- Run the `Define necessary functions` cell (only needs to be done once)
- Change the argument of `import_database` in the next cell to `2` and run the cell 
- Change the second and the third argument of `plot_transition` in the next cell to `1` and `3`, respectively and  run the cell

## **References**
Research into the topology of the configuration space of hard spheres is ongoing, but the references below provide some context for this project.

- Y. Baryshnikov, P. Bubenik, M. Kahle, [Min-type Morse theory for configuration spaces of hard spheres](https://doi.org/10.1093/imrn/rnt012), International Mathematics Research Notices 2014.9 (2014): 2577–2592.


## **Contributors**
The contributors to this project are (by alphabetical order of last name):

- Ozan Ericok (oericok@ucdavis.edu)
- Kashyap Ganesan (kganesan@ucdavis.edu)
- Jeremy Mason (jkmason@ucdavis.edu)

Please contact Jeremy Mason (jkmason@ucdavis.edu) for more information.

## **License**
The functions included in this archive are licenced under the [GNU General
Public License, Version 3](https://www.gnu.org/licenses/gpl-3.0.en.html).

## **Acknowledgements**
This material is based upon work supported by the National Science Foundation under Grant No. 1839370. Any opinions, findings, and conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the National Science Foundation.









In [8]:
#@title Define necessary functions
#Defines the import_database function
import pandas as pd
from google.colab import data_table

def import_database(n_spheres):
  filename = 'https://raw.githubusercontent.com/burakericok/hard_sphere_transitions/master/zts' + str(n_spheres) + '.csv'
  all_transitions = pd.read_csv(filename);
  return all_transitions

#Defines the plot_crit function
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
import operator

def plot_transition(all_transitions, initial, final):

  # Make sure the initial is less than final
  if initial>final:
    tmp = final
    final = initial
    initial = tmp

  zts = all_transitions.loc[((all_transitions['initial'] == initial) & (all_transitions['final'] == final))];
  x0s = zts[zts.columns[zts.columns.to_series().str.contains('coords')]]
  radii = zts[zts.columns[zts.columns.to_series().str.contains('radii')]]

  # constants
  n_spheres = np.int(np.size(x0s,1)/3);

  phi = np.linspace(0, 2*np.pi)
  theta = np.linspace(-np.pi/2, np.pi/2)
  phi, theta = np.meshgrid(phi, theta)

  x_ref = np.cos(theta) * np.sin(phi) 
  y_ref = np.cos(theta) * np.cos(phi)
  z_ref = np.sin(theta)
  n0 = np.size(x_ref,0)
  n1 = np.size(x_ref,1)

  epsilon = 1e-8
  inc = 0
  cmap = plt.get_cmap("Pastel1")
  colorscale = [];
  for i in range(9):
      colorscale.append([inc/(n_spheres-1), 'rgb' + str(tuple(map(operator.sub, cmap(i)[0:3], (epsilon,epsilon,epsilon))))])
      inc = inc + 1
  cmap = plt.get_cmap("Pastel2")
  for j in range(8):
      colorscale.append([inc/(n_spheres-1), 'rgb' + str(tuple(map(operator.sub, cmap(j)[0:3], (epsilon,epsilon,epsilon))))])
      inc = inc + 1
  colorscale = colorscale[0:n_spheres]
      
  v1 = np.array([1., 0., 0.]);
  v2 = np.array([0., 1., 0.]);
  v3 = np.array([0.5, 0.5, -1. / np.sqrt(2.)]);
  v4 = np.array([-0.5, 0.5, -1. / np.sqrt(2.)]);
  v5 = np.array([0.5, -0.5, -1. / np.sqrt(2.)]);
  v6 = np.array([-0.5, -0.5, -1. / np.sqrt(2.)]);
  v7 = v3 + v6;
  v8 = v1 + v2;
  v9 = v1 - v2;
  shift = [v1, -v1, v2, -v2, v3, -v3, v4, -v4, v5, -v5, v6, -v6, v7, -v7, v8, -v8, v9, -v9];

  frames = [];
  for k in range(np.size(x0s,0)):
      x0 = (x0s.iloc[k,:])
      r0 = (radii.iloc[k]).radii
      
      ximages = np.zeros((np.size(shift,0)+1,3*n_spheres));
      ximages[0] = x0;
      for i in range(np.size(shift,0)):
          ximages[i+1] = ximages[0] + np.matlib.repmat(shift[i],1,n_spheres)
          
          
      ximages2 = np.reshape(ximages,(np.int(np.size(ximages,0)*n_spheres),3))
      d = np.zeros((np.int(np.size(ximages2,0)*(np.size(ximages2,0)-1)/2),3));
      inc = 0;
      i = 0;
      while (i<np.size(ximages2,0)):
          j=i+1;
          while(j<np.size(ximages2,0)):
              d[inc] = [i, j, np.linalg.norm(ximages2[i]-ximages2[j])];
              inc = inc+1;
              j=j+1;
          i = i+1;

      edge_list = d[d[:,2] < 2.001*r0,:];
      pairs = edge_list[:,[0,1]];
      
      #create the coordinate list for the lines
      x_lines = list()
      y_lines = list()
      z_lines = list()
      for p in pairs:
          for i in range(2):
              x_lines.append(ximages2[np.int(p[i]),0])
              y_lines.append(ximages2[np.int(p[i]),1])
              z_lines.append(ximages2[np.int(p[i]),2])
          x_lines.append(None)
          y_lines.append(None)
          z_lines.append(None)
          
      traces = []
      for i in range(n_spheres):   
          traces.append(
              go.Scatter3d(
                  x = ximages[:,3*i],
                  y = ximages[:,3*i+1],
                  z = ximages[:,3*i+2],
                  mode = 'markers',
                  marker = dict(
                      size = 10,
                      color = colorscale[i][1],              
                      colorscale = colorscale,   
                      cmin = 0.,
                      cmax = 1.,
                      opacity = 1.0
                  )
              )
          )
      
      traces.append(
          go.Scatter3d(
              x = x_lines,
              y = y_lines,
              z = z_lines,
              mode = 'lines',
              line = dict(
                  color = 'black',
                  width = 2
              )
          )
      )
              
      frames.append(
          dict(
              data = traces,
              #traces = [0],
              name = 'frame{}'.format(k)    
          )
      )
          
          
  t = np.linspace(1, np.size(x0s,0), np.size(x0s,0))
  sliders = [
      dict(
          steps = [
              dict(
                  method = 'animate',#Sets the Plotly method to be called when the slider value is changed.
                  args = [
                      [ 'frame{}'.format(k) ],#Sets the arguments values to be passed to the Plotly method set in method on slide
                      dict( 
                          mode = 'immediate',
                          frame = dict( duration=50, redraw=False ),
                          transition = dict(duration=0)
                      )
                ],
                label = '{:.2f}'.format(t[k])
                              
              ) for k in range(t.shape[0])], 
          transition = dict(duration=0),
          x = 0,#slider starting position  
          y = 0, 
          currentvalue = dict(
              font = dict(size=12), 
              prefix = 'Bead # : ', 
              visible = True, 
              xanchor = 'center'
          ),  
          len = 1.0
      )#slider length)
  ]

  layout = dict(
      title = 'Transition from crit ' + str(initial) + ' to crit ' + str(final),
      autosize = False,
      width = 600,
      height = 600,
      showlegend = False,
      scene = dict(
          camera = dict(eye=dict(x=1.25, y=0.9, z=1.1)),
          aspectratio = dict(x=1, y=1, z=1),
      #xaxis=dict(axis),
      #yaxis=dict(axis), 
      #zaxis_range=[zmin, zmax]
      ),          
      updatemenus = [
          dict(
              type = 'buttons', 
              showactive = False,
              y = 0,
              x = 1.15,
              xanchor = 'right',
              yanchor = 'top',
              pad = dict(t=0, r=10),
              buttons = [
                  dict(
                      label = 'Play',
                      method = 'animate',
                      args = [
                          None, 
                          dict(
                              frame = dict(duration=20,redraw=True),
                              transition = dict(duration=0),
                              fromcurrent = True,
                              mode = 'immediate'
                          )
                      ]
                  )
              ]
          )
      ],
      sliders = sliders
  )

  fig = go.Figure(data=traces, layout=layout, frames=frames)
  fig.show()

In [4]:
all_transitions = import_database(2);

In [ ]:
plot_transition(all_transitions,1,4)